DB: Analytical solution test 2
=====

Comparison of a solutions obtained via ``underworld2`` and analytic solutions provided by Burstedde et al (2013).

**References**

1. *Burstedde, C., Stadler, G., Alisic, L., Wilcox, L., Tan, E., Gurnis, M., and Ghattas, O.*, Large- scale adaptive mantle convection simulation, GJI, 192, 889–906, 2013. http://gji.oxfordjournals.org/content/192/3/889.full.pdf+html}



In [ ]:
import underworld as uw
import math
from underworld import function as fn
import glucifer

import numpy as np
import matplotlib.pyplot as pyplot
import matplotlib.pylab as pylab

Setup mesh conditions
----

Change resolution parameters

In [ ]:
meshX = 16
meshY = 16
meshZ = 16

Create mesh objects using the same parameters as Dohrmann and Bochev 2004

In [ ]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q2/dQ1"), 
                                 elementRes  = (meshX, meshY, meshZ), 
                                 minCoord    = (0., 0., 0.), 
                                 maxCoord    = (1., 1., 1.))

velocityField = uw.mesh.MeshVariable(mesh,3)
pressureField = uw.mesh.MeshVariable(mesh.subMesh,1)

**Setup analytic functions**

Functions consist of the body force, viscosity and velocity field inputs as well as the resulting pressure, stress and strain rate fields.


In [ ]:
solA = fn.analytic.SolDB3d( Beta = 4. )

**Setup initial conditions**

Set initial velocity field to be the analytical solution, but set the initial pressure field to zero. The pressure field calculated by the Stokes solver will be compared to the analytical solution at the end.

Note that the boundary condition values are also set at this point.

In [ ]:
vel=solA.fn_velocity
velocityField.data[:] = vel.evaluate(mesh.data)
pressureField.data[:] = 0.

**Plot the vertical component of the body force solution**


In [ ]:
force = glucifer.Figure()
force.append( glucifer.objects.Surface(mesh, solA.fn_bodyforce[2]) )
force.show()

**Plot the magnitude of the velocity field**

In [ ]:
velMag = glucifer.Figure()
velMag.append( glucifer.objects.Surface(mesh, fn.math.dot(velocityField,velocityField)) )
velMag.show()

**Set dirichlet boundary conditions on all walls**

In [ ]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
kWalls = mesh.specialSets["MinK_VertexSet"] + mesh.specialSets["MaxK_VertexSet"]
allWalls = iWalls + jWalls + kWalls

BC = uw.conditions.DirichletCondition( variable        = velocityField, 
                                       indexSetsPerDof = (allWalls, allWalls, allWalls) )

Setup Stokes system
---

**Change here to test other solver options**

In [ ]:
stokesSystem = uw.systems.Stokes(velocityField = velocityField,
                                 pressureField = pressureField,
                                 fn_viscosity  = solA.fn_viscosity,
                                 fn_bodyforce  = solA.fn_bodyforce,
                                 conditions    = [BC,])

solver = uw.systems.Solver( stokesSystem )

**Solve Stokes system**

This will solve for the pressure given the velocity, viscosity and body force.

In [ ]:
solver.options.scr.ksp_rtol = 1.e-5 
solver.set_inner_method(solve_type="lu")
solver.solve()

Compare ``underworld`` and analytical solutions
----

**Plot analytic solution to the pressure field**

In [ ]:
pA = glucifer.Figure()
pA.append( glucifer.objects.Surface(mesh, solA.fn_pressure) )
pA.show()

**Plot difference between solved pressure field and analytic solution**

In [ ]:
pDiff = glucifer.Figure()
pDiff.append( glucifer.objects.Surface(mesh, solA.fn_pressure-pressureField ) )
pDiff.show()

**Examine error across a single line**

Look at the error in the pressure field for $x=0.85$ and $z=0.0$, scaled to fit along side the exact and numerical solution.

In [ ]:
if(uw.nProcs()==1):
    uw.matplotlib_inline()
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pyplot.ion() # needed to ensure pure python jobs do now hang on show()
    N=201
    a=np.ndarray(shape=(N,3))
    a[:,0]=0.85*np.ones(N)
    a[:,1]=np.linspace(0,1,N)
    a[:,2]=np.linspace(0,1,N)
    ax=a[:,0]
    pex=solA.fn_pressure.evaluate(a)[:,0]
    y=a[:,1]
    pfd=pressureField.evaluate(a)[:,0]
    pyplot.plot(y,pfd,label='Numerical')
    pyplot.plot(y,pex,label='Exact')
    pyplot.plot(y,100*(pfd-pex),label='Error (x 100)')
    pyplot.legend()
    
# the following was the parallel error check
# using evaluate_global. It produced funny results
# I won't investigate as this analytic sol is moving into broken

#     N=801
#     a=np.ndarray(shape=(N,3))
#     a[:,0]=0.85*np.ones(N)
#     a[:,1]=np.linspace(0,1,N)
#     a[:,2]=0.0
#     ax=a[:,1]
#     pex=solA.fn_pressure.evaluate_global( a[:,])
#     y=a[:,1]
#     pfd=pressureField.evaluate_global(a[:,])
#     pyplot.plot(y,pfd,label='Numerical')
#     pyplot.plot(y,pex,label='Exact')
#     pyplot.plot(y,100*(pfd-pex),label='Error (x 100)')
#     pyplot.legend()
#     pyplot.savefig('Pressure_Error.png')

Global error measures
-----

Work out the global rms values for the velocity error (should be very low) and the pressure error.

In [ ]:
v_err = uw.utils.Integral( fn   = fn.math.dot(solA.fn_velocity-velocityField, solA.fn_velocity-velocityField), 
                           mesh = mesh )
v_sol = uw.utils.Integral( fn   = fn.math.dot(solA.fn_velocity, solA.fn_velocity), 
                           mesh = mesh )
p_err = uw.utils.Integral( fn   = fn.math.dot(solA.fn_pressure-pressureField, solA.fn_pressure-pressureField), 
                           mesh = mesh )
p_sol = uw.utils.Integral( fn   = fn.math.dot(solA.fn_pressure, solA.fn_pressure), 
                           mesh = mesh )

area    = uw.utils.Integral( fn = 1., mesh = mesh )
totalArea = area.evaluate()[0]

rms_velocity_err = math.sqrt( v_err.evaluate()[0]/totalArea )
rms_velocity_rel = math.sqrt( v_err.evaluate()[0]/v_sol.evaluate()[0] )
rms_pressure_err = math.sqrt( p_err.evaluate()[0]/totalArea )
rms_pressure_rel = math.sqrt( p_err.evaluate()[0]/p_sol.evaluate()[0] )

In [ ]:
if uw.rank()==0:
    print("Pressure error = {0:.6f}%; relative error = {1:.6f}%"
          .format(rms_pressure_err*100, rms_pressure_rel*100))
    print("Velocity error = {0:.6f}%; relative error = {1:.6f}%"
          .format(rms_velocity_err*100, rms_velocity_rel*100))

In [ ]:
# errors on OSX @ commit 7b4a45bd5cc935e5553707719c4d37f3b53890eb
# petsc 3.6.3_4, open-mpi/1.10.2
rms_pressure_err_expected = 0.0032669226502844537
rms_pressure_rel_expected = 0.014295521589953772
rms_velocity_err_expected = 4.948183259946877e-06
rms_velocity_rel_expected = 1.188653968407299e-06

In [ ]:
tolerance = 1.01
if rms_pressure_err/rms_pressure_err_expected > tolerance: raise RuntimeError("Pressure absolute error outside tolerance")
if rms_pressure_rel/rms_pressure_rel_expected > tolerance: raise RuntimeError("Pressure relative error outside tolerance")
if rms_velocity_err/rms_velocity_err_expected > tolerance: raise RuntimeError("Velocity absolute error outside tolerance")
if rms_velocity_rel/rms_velocity_rel_expected > tolerance: raise RuntimeError("Velocity relative error outside tolerance")